In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
query = {  
    "queries": [
        {
            "type" : "queryText",
            "value": "temperature"
        }      
    ],
    "summary": False
}

In [27]:
import httpx
headers = {'Accept': 'application/json'}

temperature = httpx.post('https://data.noaa.gov/onestop/api/search/search/collection', headers=headers, json=query).json()

In [28]:
temperature

{'data': [{'attributes': {'stagedDate': 1688288695874,
    'fileIdentifier': 'gov.noaa.ncei:SeaSurfaceTemperature',
    'title': '.Sea Surface Temperature',
    'description': 'Derived from processed satellite observations. These data are usually within 0.5 degrees Celsius uncertainty for ocean areas and 1 degree Celsius uncertainty for coastal areas.',
    'keywords': [],
    'spatialBounding': {'type': 'Polygon',
     'coordinates': [[[-97.9, 17.1],
       [-78.66, 17.1],
       [-78.66, 31.6],
       [-97.9, 31.6],
       [-97.9, 17.1]]]},
    'isGlobal': False,
    'dataFormats': [{'name': 'PNG'}],
    'dataFormat': ['PNG'],
    'links': [{'linkFunction': 'completeMetadata'}],
    'linkProtocol': [],
    'serviceLinkProtocol': [],
    'individualNames': [],
    'organizationNames': [],
    'thumbnail': 'Not provided.',
    'citeAsStatements': [],
    'crossReferences': [],
    'largerWorks': [],
    'legalConstraints': [],
    'dsmmAverage': 0.0,
    'serviceLinks': []},
   'id': '

In [35]:
import pandas as pd
temp_df = pd.json_normalize(temperature['data'])

In [6]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [7]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [8]:
%sql duckdb:///:default:

In [39]:
%%sql 

CREATE TABLE onestop_datasets AS SELECT * FROM temp_df

,Count
0,1


In [42]:
%sql DESCRIBE TABLE onestop_datasets

,column_name,column_type,null,key,default,extra
0,id,VARCHAR,YES,NaN,NaN,NaN
1,type,VARCHAR,YES,NaN,NaN,NaN
2,attributes.stagedDate,BIGINT,YES,NaN,NaN,NaN
3,attributes.fileIdentifier,VARCHAR,YES,NaN,NaN,NaN
4,attributes.title,VARCHAR,YES,NaN,NaN,NaN
5,attributes.description,VARCHAR,YES,NaN,NaN,NaN
6,attributes.keywords,INTEGER[],YES,NaN,NaN,NaN
7,attributes.spatialBounding.type,VARCHAR,YES,NaN,NaN,NaN
8,attributes.spatialBounding.coordinates,DOUBLE[][][],YES,NaN,NaN,NaN
9,attributes.isGlobal,BOOLEAN,YES,NaN,NaN,NaN


In [46]:
%sql ALTER TABLE onestop_datasets ADD COLUMN onestop_query VARCHAR

,Success


In [49]:
from datasets import async_pool_fetcher

con = duckdb.connect(database=':default:')

def duckdb_conn_wrapper(duckdb_con, table):
    def handle_results(data):
        local_con = duckdb_con.cursor()
        df = pd.json_normalize(data['data'])
        df['onestop_query'] = query['queries'][0]
        local_con.execute(f"""
            INSERT INTO {table} SELECT * FROM df
        """)
    return handle_results

handler = duckdb_conn_wrapper(con, 'onestop_datasets')


In [59]:
from datasets.async_pool_fetcher import main

await main('https://data.noaa.gov/onestop/api/search/search/collection', 80, 20, handler, query)

<coroutine object AsyncPaginator.__anext__ at 0x14181d8c0>


TypeError: 'coroutine' object is not subscriptable